In [1]:
import pickle
import io
import gensim
import nltk
import numpy as np
import pickle
import matplotlib.pyplot as plt

from collections import Counter

from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

from math import*
from nltk.stem.porter import *
from nltk.corpus import wordnet as wn

from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection  import KFold, cross_val_score
from sklearn import svm
from sklearn import datasets, linear_model, model_selection, NuSVR

from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.spatial.distance import jaccard
from scipy.stats import kendalltau

model_name="ft"

feature_vector={
    "emb":True,
    "Suffix":True,
    "POS":True
}

from IPython.display import Audio
sound_file = 'tpir-dings.mp3'

# FastText

In [3]:
def load_vectors(fname):
    reloaded_word_vectors = KeyedVectors.load_word2vec_format(fname)
    return reloaded_word_vectors

# Load pretrained word2vec model
model=load_vectors("D:\\PhD\\fastText_Embeddings\\crawl-300d-2M-subword.vec")


# Helper

In [4]:
with open('most_common_ending.pkl', 'rb') as f:
    most_common_ending = pickle.load(f)
with open('most_common_porter.pkl', 'rb') as f:
    most_common_porter = pickle.load(f)

In [5]:
def getAllPostfix(s):
    if len(s)>3:
        return [s[-1:],s[-2:],s[-3:],s[-4:]]
    elif len(s)==3:
        return [s[-1:],s[-2:],s[-3:]]
    elif len(s)==2:
        return [s[-1:],s[-2:]]
    elif len(s)==1:
        return [s]
    else:
        return [""]

def getPostfixVec(s,mc,dim=100):
    vec=[0]*dim
    Post=getAllPostfix(s)
    #Post=Func(s)
    for i in range(dim):
        if mc[i] in Post:
            vec[i]=1
    return vec

def getPorterVec(s,mc,dim=150):
    vec=[0]*dim
    Post=getPorterRest(s)
    print(mc)
    for i in range(dim):
        if mc[i] == Post:
            vec[i]=1
    return vec
    
endingDir={}
endingList=[]

In [6]:
stemmer = PorterStemmer()
def getPorterRest(s):
    stem=stemmer.stem(s)
    ret=""
    for k in range(len(s)-len(stem)):
        ret+=(s[len(stem)+k])
    return ret
porterList=[]

In [7]:
def getVecFromPos(s):
    vec=[]
    if s=='a':
        vec=[1,0,0,0,0,0]
    if s=='n':
        vec=[0,1,0,0,0,0]
    if s=='p':
        vec=[0,0,1,0,0,0]
    if s=='r':
        vec=[0,0,0,1,0,0]
    if s=='s':
        vec=[0,0,0,0,1,0]
    if s=='v':
        vec=[0,0,0,0,0,1]
    return vec#np.asarray(vec)        

In [8]:
def getPos(w):
    poss=[] 
    posSum=0
    posfreqs = {'n':0,'v':0,'a':0,'p':0,'r':0,'s':0}
    found=False
    for syns in wn.synsets(w):
        p = syns.pos()
        if p not in poss:
            poss.append(p)
        count = posfreqs.get(p,0)
        for l in syns.lemmas():
            count += l.count()
            if count>0:
                found=True
        posfreqs[p] = count
    if found:
        for fr in posfreqs:
            posSum+=posfreqs[fr]
        pos=[posfreqs['n']/posSum,posfreqs['v']/posSum,posfreqs['a']/posSum,posfreqs['p']/posSum,posfreqs['r']/posSum,posfreqs['s']/posSum]
    else:
        tags = nltk.pos_tag([w])
        tag = tags[0][1]
        if tag[0] == 'R':
            pos = 'r'
            pos=[0,0,0,0,1,0]
        elif tag[0] == 'J':
            pos = 'a'
            pos=[0,0,1,0,0,0]
        elif tag[0] == 'V':
            pos = 'v'
            pos=[0,1,0,0,0,0]
        elif tag == 'IN':
            pos = 'p'
            pos=[0,0,0,1,0,0]
        else:
            pos = 'n'
            pos=[1,0,0,0,0,0]
    return pos


def transformScale(old_value, old_min = 1, old_max = 7, new_min = 1, new_max = 5):
    new_value = ( (old_value - old_min) / (old_max - old_min) ) * (new_max - new_min) + new_min
    if new_value>new_max:
        print("new_value > max")
    return new_value

In [9]:
def jaccard_similarity(x,y,verbose=0):
  
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    if verbose>0:
        print("\n Intersection Cardinality: "+str(intersection_cardinality))
        print("Unity Cardinality: "+str(union_cardinality))
        print("\n")
    return intersection_cardinality/float(union_cardinality)


def getFeature(w,embeddingmodel=model,emb=True,POS=True,Post=True,verbose=0):
    emb=feature_vector["emb"]
    POS=feature_vector["POS"]
    Post=feature_vector["Suffix"]
    vect=[]
    vect=np.asarray(vect)
    if verbose>0:
        print(feature_vector)
    vect=np.append(vect,model[w])
    vect=np.append(vect,getPos(w))
    vect=np.append(vect,getPostfixVec(w,most_common_ending,dim=200))
    return vect

def getPrediction(vec,trained_model):
    return trained_model.predict([vec])

def get_r2_numpy_corrcoef(x, y):
    return np.corrcoef(x, y)[0, 1]**2

def getOwnR2(x,y):
    x=np.asarray(x)
    y=np.asarray(y)
    x_mean=x.mean()
    y_mean=y.mean()
    defined_var=0
    total_var=0
    undefined_var=0
    for i in range(len(y)):
        undefined_var+= (x[i]-y[i])**2
        defined_var += (y[i]-x_mean)**2
        total_var += (x[i]-x_mean)**2
        
    r2_val=defined_var/total_var
    r2_val2=1-(undefined_var/total_var)
    return r2_val,r2_val2
        

def evalModel(listA,listB,own=False,):
    r_score=r2_score(listA,listB)
    p_score=pearsonr(listA,listB)
    s_score=spearmanr(listA,listB)
    np_score=get_r2_numpy_corrcoef(listA,listB)
    mse_score=mean_squared_error(listA,listB)
    r2_o,r2_o2=getOwnR2(listA,listB)
    tau_score=kendalltau(listA,listB)
    if own:
        print("\n########  OWN  ###########\n")
        print(" r2: "+str(r2_o))
        print("own r2_2: "+str(r2_o2))
    print("\n######## CUSTOM ###########\n")
    print("Pearson: "+str(p_score))
    print("Kendall: "+str(tau_score))
    print("Spearman: "+str(s_score))
    print("Pearson**2: "+str(np_score))
    print("R^2 Score: "+str(r_score))
    print("MSE: "+str(mse_score))
    print("\n##########################\n")

In [10]:
def loadMT40K(model_para):
        
    words = []
    
    x=[]
    y=[]
    reg=[]
    numKeyError=0
    MT40KnoEmb=[]
    
    with open('cor.csv', 'r') as f:
        header = f.readline()
        for line in f.readlines():
            words.append(line.strip().split(';'))
         
         
    fout = open('concreteness_pos.txt', 'w')
        
    i=0
    for w in words:
        if int(w[1]) == 0:
            ##################################

            for wi in getAllPostfix(w[0]):

                endingList.extend([wi])
            porterList.append(getPorterRest(w[0]))
            ########
           

            if w[1]=="0": #Bigramm=0 == only one word
                corVal=w[3].replace(",",".") # make floats
                corStd=w[4].replace(",",".")
                try:            
                    vect=getFeature(w[0])     
                    x.append(np.asarray(vect))
                    y.append(float(corVal))
                except KeyError:
                    numKeyError+=1
                    MT40KnoEmb.append(w[0])

    fout.close()
    x=np.asarray(x)
    y=np.asarray(y)
    print("no Embedding: "+str(numKeyError))
    print("Vector Size: ",len(x))
    return x, y

# build FeatureVector

In [11]:
words = []

vectorDic={}
concDic={}
dat=[]
tar=[]
reg=[]
std=[]
tarstd=[]
numKeyError=0
raw=[]
MT40K=[]
MT40K_allWords=[]
MT40KnoEmb=[]
MT40K_twophrase=[]

with open('cor.csv', 'r') as f:
    header = f.readline()
    for line in f.readlines():
        words.append(line.strip().split(';'))
     
     
fout = open('concreteness_pos.txt', 'w')

######################
CP_wordlist=[]
TWP_wordlist=[]
CPExt_wordlist=[]

skipped_content=0
for w in words:
    if int(w[1]) == 0:
        MT40K_allWords.append(w[0])
    else:
        MT40K_twophrase.append(w[0])
    if w[0] in CP_wordlist or w[0] in TWP_wordlist or w[0] in CPExt_wordlist:
        skipped_content+=1
        continue
    if int(w[1]) == 0:
        posfreqs = {}
        for syns in wn.synsets(w[0]):
            p = syns.pos()
            count = posfreqs.get(p,0)
            for l in syns.lemmas():
                count += l.count()
            posfreqs[p] = count
        posfreqs = sorted(posfreqs.items(),key = lambda x:x[1],reverse = True)
        if len(posfreqs) > 0:
            pos = posfreqs[0][0]
        else:
            tags = nltk.pos_tag([w[0]])
            tag = tags[0][1]
            if tag[0] == 'R':
                pos = 'r'
            elif tag[0] == 'J':
                pos = 'a'
            elif tag[0] == 'V':
                pos = 'v'
            elif tag == 'IN':
                pos = 'p'
            else:
                pos = 'n'
        
        w1 = [w[0],pos]+w[1:]
        
        for wi in getAllPostfix(w[0]):

            endingList.extend([wi])
        porterList.append(getPorterRest(w[0]))
        ########
              
        if w1[2]=="0": #Bigramm=0 == only one word
            corVal=w1[3].replace(",",".") # make floats
            corStd=w1[4].replace(",",".")
            try:
            
                #vectorDic[w1[0]]=model[w1[0]]
                concDic[w1[0]]=corVal
                
                #Feature Vector
                vect=getFeature(w1[0],emb=True,POS=True,Post=True)
                              
                #Save for Eval
                MT40K.append((w1[0],getPos(w1[0]),corVal,corStd)) #1=word,2=pos,3=conc,4=std

                
                dat.append(np.asarray(vect))
                tar.append(float(corVal))
                std.append(float(corStd))
                tarstd.append([float(corVal),float(corStd)])
                raw.append(w1)
            except KeyError:
                numKeyError+=1
                MT40KnoEmb.append(w1[0])               
                
fout.close()
dat=np.asarray(dat)
tar=np.asarray(tar)
std=np.asarray(std)
print("allWords: ",len(words))
print("skipped_content: ",skipped_content)
print("no embedding: ",numKeyError)
print("\nMT40k:",len(MT40K)) 
print("MT40k all 1-phrase words: ",len(MT40K_allWords))
print("MT40k 2-Phrase: ",len(MT40K_twophrase))
print("MT40k Union: ",len(MT40K_allWords)+len(MT40K_twophrase))


Audio(sound_file, autoplay=True)

allWords:  39954
skipped_content:  0
no embedding:  1584

MT40k: 35474
MT40k all 1-phrase words:  37058
MT40k 2-Phrase:  2896
MT40k Union:  39954


In [12]:
c_p =Counter(porterList).most_common(200)
c_e =Counter(endingList).most_common(200)

most_common_porter=[]
most_common_ending=[]
for t in c_p:
    most_common_porter.append(t[0])
for t in c_e:
    most_common_ending.append(t[0])
    
with open('most_common_porter.pkl', 'wb') as f:
    pickle.dump(most_common_porter, f)
with open('most_common_ending.pkl', 'wb') as f:
    pickle.dump(most_common_ending, f)



# Eval Training

# Pearson CV

In [16]:
reg = svm.SVR(gamma=0.01,C=1.0,kernel='rbf',verbose=2)

def corr(y, y_pred, **kwargs):
    ret=np.corrcoef(y, y_pred)[0, 1]#**2
    return ret

scor=make_scorer(corr, greater_is_better=True, needs_proba=False, needs_threshold=False)

score=cross_val_score(reg,  dat, tar, cv=8,n_jobs=-1,scoring=scor,verbose=2)


print(score)
print("Pearson : %0.4f (+/- %0.4f)" % (score.mean(), score.std() * 2))
Audio(sound_file, autoplay=True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed: 11.7min remaining: 35.1min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 11.7min finished


[0.87979672 0.87239161 0.86818968 0.88435498 0.86151988 0.86816019
 0.85581788 0.87858561]
Pearson : 0.8711 (+/- 0.0180)


# Spearman CV

In [17]:
reg = svm.SVR(gamma=0.01,C=1.0,kernel='rbf',verbose=0)

def corrTau(y, y_pred, **kwargs):
    ret=s_score=spearmanr(y,y_pred).correlation
    #ret=kendalltau(y,y_pred).correlation
    return ret

scorTau=make_scorer(corrTau, greater_is_better=True, needs_proba=False, needs_threshold=False)

score=cross_val_score(reg,  dat, tar, cv=8,n_jobs=-1,scoring=scorTau,verbose=2)


print(score)
print("Tau : %0.4f (+/- %0.4f)" % (score.mean(), score.std() * 2))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed: 11.5min remaining: 34.6min
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 11.6min finished


[0.88147282 0.85678241 0.86247198 0.87368007 0.8545626  0.85830122
 0.86187818 0.86026379]
Tau : 0.8637 (+/- 0.0172)


In [18]:
Audio(sound_file, autoplay=True)

# Train the final model on all data.

In [19]:
print("Start Training")
reg = svm.SVR(gamma=0.01,C=1.0,kernel='rbf',verbose=10)

reg.fit(dat,tar)
print("Finished training")

#save model
pickle.dump(reg, open("Concreteness.model", 'wb'))

Audio(sound_file, autoplay=True)

Start Training
[LibSVM]finished training
